<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center> Authors: [Maria Sumarokova](https://www.linkedin.com/in/mariya-sumarokova-230b4054/), senior data scientist/analyst at Veon, and [Yury Kashnitsky](https://www.linkedin.com/in/festline/), data scientist at Mail.Ru Group. <br>Translated and edited by Gleb Filatov, Aleksey Kiselev,<br> [Anastasia Manokhina](https://www.linkedin.com/in/anastasiamanokhina/), [Egor Polusmak](https://www.linkedin.com/in/egor-polusmak/), and [Yuanyuan Pao](https://www.linkedin.com/in/yuanyuanpao/) <br>All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

# <center> Assignment #3 (demo)
## <center>  Decision trees with a toy task and the UCI Adult dataset 

Please fill in the answers in the [web-form](https://docs.google.com/forms/d/1wfWYYoqXTkZNOPy1wpewACXaj2MZjBdLOL58htGWYBA/edit).

Let's start by loading all necessary libraries:

In [259]:
import numpy as np
import pandas as pd
import seaborn as sns
import collections

from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 8)

### Part 1. Toy dataset "Will They? Won't They?"

Your goal is to figure out how decision trees work by walking through a toy problem. While a single decision tree does not yield outstanding results, other performant algorithms like gradient boosting and random forests are based on the same idea. That is why knowing how decision trees work might be useful.

We'll go through a toy example of binary classification - Person A is deciding whether they will go on a second date with Person B. It will depend on their looks, eloquence, alcohol consumption (only for example), and how much money was spent on the first date.

#### Creating the dataset

In [2]:
# Create dataframe with dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

# Some feature values are present in train and absent in test and vice-versa.
def intersect_features(train, test):
    common_feat = list( set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]

In [3]:
features = ['Looks', 'Alcoholic_beverage','Eloquence','Money_spent']

#### Training data

In [4]:
df_train = {}
df_train['Looks'] = ['handsome', 'handsome', 'handsome', 'repulsive',
                         'repulsive', 'repulsive', 'handsome'] 
df_train['Alcoholic_beverage'] = ['yes', 'yes', 'no', 'no', 'yes', 'yes', 'yes']
df_train['Eloquence'] = ['high', 'low', 'average', 'average', 'low',
                                   'high', 'average']
df_train['Money_spent'] = ['lots', 'little', 'lots', 'little', 'lots',
                                  'lots', 'lots']
df_train['Will_go'] = LabelEncoder().fit_transform(['+', '-', '+', '-', '-', '+', '+'])

df_train = create_df(df_train, features)
df_train

,Will_go,Looks_handsome,Looks_repulsive,Alcoholic_beverage_no,Alcoholic_beverage_yes,Eloquence_average,Eloquence_high,Eloquence_low,Money_spent_little,Money_spent_lots
0,0,1,0,0,1,0,1,0,0,1
1,1,1,0,0,1,0,0,1,1,0
2,0,1,0,1,0,1,0,0,0,1
3,1,0,1,1,0,1,0,0,1,0
4,1,0,1,0,1,0,0,1,0,1
5,0,0,1,0,1,0,1,0,0,1
6,0,1,0,0,1,1,0,0,0,1


#### Test data

In [5]:
df_test = {}
df_test['Looks'] = ['handsome', 'handsome', 'repulsive'] 
df_test['Alcoholic_beverage'] = ['no', 'yes', 'yes']
df_test['Eloquence'] = ['average', 'high', 'average']
df_test['Money_spent'] = ['lots', 'little', 'lots']
df_test = create_df(df_test, features)
df_test

,Looks_handsome,Looks_repulsive,Alcoholic_beverage_no,Alcoholic_beverage_yes,Eloquence_average,Eloquence_high,Money_spent_little,Money_spent_lots
0,1,0,1,0,1,0,0,1
1,1,0,0,1,0,1,1,0
2,0,1,0,1,1,0,0,1


In [6]:
# Some feature values are present in train and absent in test and vice-versa.
y = df_train['Will_go']
df_train, df_test = intersect_features(train=df_train, test=df_test)
df_train

,Money_spent_little,Looks_handsome,Alcoholic_beverage_yes,Looks_repulsive,Money_spent_lots,Alcoholic_beverage_no,Eloquence_average,Eloquence_high
0,0,1,1,0,1,0,0,1
1,1,1,1,0,0,0,0,0
2,0,1,0,0,1,1,1,0
3,1,0,0,1,0,1,1,0
4,0,0,1,1,1,0,0,0
5,0,0,1,1,1,0,0,1
6,0,1,1,0,1,0,1,0


In [7]:
df_test

,Money_spent_little,Looks_handsome,Alcoholic_beverage_yes,Looks_repulsive,Money_spent_lots,Alcoholic_beverage_no,Eloquence_average,Eloquence_high
0,0,1,0,0,1,1,1,0
1,1,1,1,0,0,0,0,1
2,0,0,1,1,1,0,1,0


#### Draw a decision tree (by hand or in any graphics editor) for this dataset. Optionally you can also implement tree construction and draw it here.

1\. What is the entropy $S_0$ of the initial system? By system states, we mean values of the binary feature "Will_go" - 0 or 1 - two states in total.

In [26]:
entropy = lambda x: (-x[0]*np.log(x[0]) - x[1]*np.log([1]))[0]
s_0 = entropy(y.value_counts(normalize=True))
s_0

0.319780450248813

2\. Let's split the data by the feature "Looks_handsome". What is the entropy $S_1$ of the left group - the one with "Looks_handsome". What is the entropy $S_2$ in the opposite group? What is the information gain (IG) if we consider such a split?

In [54]:
# you code here
left = y[df_train["Looks_handsome"] >= 0.5]
right = y[df_train["Looks_handsome"] < 0.5]

s_1 = entropy(left.value_counts(normalize=True))
s_2 = entropy(right.value_counts(normalize=True))

In [55]:
inforamtion_gain = s_0 - (len(left)/len(y))*s_1 - (len(right)/len(y))*s_2
inforamtion_gain

0.039543520816891276

#### Train a decision tree using sklearn on the training data. You may choose any depth for the tree.

In [31]:
# you code here
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(df_train, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf='deprecated', min_samples_split=2,
            min_weight_fraction_leaf='deprecated', presort=False,
            random_state=None, splitter='best')

#### Additional: display the resulting tree using graphviz. You can use pydot or [web-service](https://www.coolutils.com/ru/online/DOT-to-PNG) dot2png.

In [52]:
# you code here

export_graphviz(tree, feature_names=df_train.columns, out_file="../../img/tree_1_assigment_3.dot", filled=True)
!dot -Tpng '../../img/tree_1_assigment_3.dot' -o '../../img/tree_1_assigment_3.png'

<img src="../../img/tree_1_assigment_3.png">

### Part 2. Functions for calculating entropy and information gain.

Consider the following warm-up example: we have 9 blue balls and 11 yellow balls. Let ball have label **1** if it is blue, **0** otherwise.

In [79]:
balls = [1 for i in range(9)] + [0 for i in range(11)]
balls

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

<img src = '../../img/decision_tree3.png'>

Next split the balls into two groups:

<img src = '../../img/decision_tree4.png'>

In [80]:
# two groups
balls_left  = [1 for i in range(8)] + [0 for i in range(5)] # 8 blue and 5 yellow
balls_right = [1 for i in range(1)] + [0 for i in range(6)] # 1 blue and 6 yellow
balls_left, balls_right

([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0])

#### Implement a function to calculate the Shannon Entropy

In [121]:
import scipy as sc

def ent(data):
    p_data = data.value_counts()/len(data)
    entropy = sc.stats.entropy(p_data)  
    return entropy

In [138]:
# def count_classes(a_list):
#     u_classes = set(a_list)
#     all_probability = []
    
#     for cls in u_classes:
#         cls_fraction = a_list.count(cls) / len(a_list)
#         all_probability.append(cls_fraction)
#     return all_probability

def entropy(a_list):
    all_probability = np.unique(np.array(a_list), return_counts=True)[1] / len(a_list)
    entropy = -np.sum(all_probability * np.log(all_probability)) 
    return entropy

Tests

In [135]:
print(entropy(balls)) # 9 blue and 11 yellow
print(entropy(balls_left)) # 8 blue and 5 yellow
print(entropy(balls_right)) # 1 blue and 6 yellow
print(entropy([1,2,3,4,5,6])) # entropy of a fair 6-sided dice

0.6881388137135884
0.666278442414676
0.410116318288409
1.7917594692280547


3\. What is the entropy of the state given by the list **balls_left**?

4\. What is the entropy of a fair dice? (where we look at a dice as a system with 6 equally probable states)?

In [203]:
# information gain calculation
def information_gain(root, left, right):
    ''' root - initial data, left and right - two partitions of initial data'''
    left_fraction = len(left)/len(root)
    right_fraction = len(right)/len(root)
    try:
        inf_gain =  entropy(root) - left_fraction*entropy(left) - right_fraction*entropy(right)
    except Exception as e:
        print("Problem here!")
    
    return inf_gain

In [159]:
np.arange(10)

9

5\. What is the information gain from splitting the initial dataset into **balls_left** and **balls_right** ?

In [140]:
# you code here
information_gain(balls, balls_left, balls_right)

0.11151711474310586

In [156]:
np.arange(8).reshape(2,4)[:, 0]

array([0, 4])

In [237]:
def best_feature_to_split(X, y, info=False):
    '''Outputs information gain when splitting on best feature'''
    n_features = X.shape[1]
    max_inf = {
        "feature_index": -1, 
        "max_inf_gain": -1, 
        "split_value": -1
    }
   
    for feature in range(n_features):
        column = X[:, feature]
        inf_by_col = []
        for index in range(column.shape[0] - 1):
            split = column[index] + column[index + 1] / 2.0
            left = y[column <= split]
            right = y[column > split]
            if info:
                print("Feature_index: {}, Split value: {}, Information_gain: {}" \
                      .format(feature, split, information_gain(y, left, right)))
    
            inf_by_col.append(information_gain(y, left, right))
    
        inf_by_col = np.array(inf_by_col)
        
        if inf_by_col[inf_by_col.argmax()] > max_inf["max_inf_gain"]:
            max_inf["max_inf_gain"] = inf_by_col[inf_by_col.argmax()]
            max_inf["feature_index"] = feature
            max_inf["split_value"] = column[inf_by_col.argmax()]
    
    return max_inf

In [238]:
best_feature_to_split(df_train.values, y, info=True)

Feature_index: 0, Split value: 0.5, Information_gain: 0.32547780217319466
Feature_index: 0, Split value: 1.0, Information_gain: 0.0
Feature_index: 0, Split value: 0.5, Information_gain: 0.32547780217319466
Feature_index: 0, Split value: 1.0, Information_gain: 0.0
Feature_index: 0, Split value: 0.0, Information_gain: 0.32547780217319466
Feature_index: 0, Split value: 0.0, Information_gain: 0.32547780217319466
Feature_index: 1, Split value: 1.5, Information_gain: 0.0
Feature_index: 1, Split value: 1.5, Information_gain: 0.0
Feature_index: 1, Split value: 1.0, Information_gain: 0.0
Feature_index: 1, Split value: 0.0, Information_gain: 0.08878194993480437
Feature_index: 1, Split value: 0.0, Information_gain: 0.08878194993480437
Feature_index: 1, Split value: 0.5, Information_gain: 0.08878194993480437
Feature_index: 2, Split value: 1.5, Information_gain: 0.0
Feature_index: 2, Split value: 1.0, Information_gain: 0.0
Feature_index: 2, Split value: 0.0, Information_gain: 0.004143433819589815
F

{'feature_index': 0, 'max_inf_gain': 0.32547780217319466, 'split_value': 0}

#### Optional:
- Implement a decision tree building algorithm by calling **best_feature_to_split** recursively
- Plot the resulting tree

### Part 3. The "Adult" dataset

#### Dataset description:

[Dataset](http://archive.ics.uci.edu/ml/machine-learning-databases/adult) UCI Adult (no need to download it, we have a copy in the course repository): classify people using demographical data - whether they earn more than \$50,000 per year or not.

Feature descriptions:

- **Age** – continuous feature
- **Workclass** –  continuous feature
- **fnlwgt** – final weight of object, continuous feature
- **Education** –  categorical feature
- **Education_Num** – number of years of education, continuous feature
- **Martial_Status** –  categorical feature
- **Occupation** –  categorical feature
- **Relationship** – categorical feature
- **Race** – categorical feature
- **Sex** – categorical feature
- **Capital_Gain** – continuous feature
- **Capital_Loss** – continuous feature
- **Hours_per_week** – continuous feature
- **Country** – categorical feature

**Target** – earnings level, categorical (binary) feature.

#### Reading train and test data

In [239]:
data_train = pd.read_csv('../../data/adult_train.csv', sep=';')

In [240]:
data_train.tail()

,Age,Workclass,fnlwgt,Education,Education_Num,Martial_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Country,Target
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [241]:
data_test = pd.read_csv('../../data/adult_test.csv', sep=';')

In [242]:
data_test.tail()

,Age,Workclass,fnlwgt,Education,Education_Num,Martial_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Country,Target
16277,39,Private,215419.0,Bachelors,13.0,Divorced,Prof-specialty,Not-in-family,White,Female,0.0,0.0,36.0,United-States,<=50K.
16278,64,NaN,321403.0,HS-grad,9.0,Widowed,NaN,Other-relative,Black,Male,0.0,0.0,40.0,United-States,<=50K.
16279,38,Private,374983.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
16280,44,Private,83891.0,Bachelors,13.0,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455.0,0.0,40.0,United-States,<=50K.
16281,35,Self-emp-inc,182148.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,60.0,United-States,>50K.


In [243]:
# necessary to remove rows with incorrect labels in test dataset
data_test = data_test[(data_test['Target'] == ' >50K.') | (data_test['Target']==' <=50K.')]

# encode target variable as integer
data_train.loc[data_train['Target']==' <=50K', 'Target'] = 0
data_train.loc[data_train['Target']==' >50K', 'Target'] = 1

data_test.loc[data_test['Target']==' <=50K.', 'Target'] = 0
data_test.loc[data_test['Target']==' >50K.', 'Target'] = 1

#### Primary data analysis

In [245]:
data_train.dtypes

Age                int64
Workclass         object
fnlwgt             int64
Education         object
Education_Num      int64
Martial_Status    object
Occupation        object
Relationship      object
Race              object
Sex               object
Capital_Gain       int64
Capital_Loss       int64
Hours_per_week     int64
Country           object
Target             int64
dtype: object

In [250]:
data_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,32561.0,38.581647,13.640433,17.0,28.0,37.0,48.0,90.0
fnlwgt,32561.0,189778.366512,105549.977697,12285.0,117827.0,178356.0,237051.0,1484705.0
Education_Num,32561.0,10.080679,2.572720,1.0,9.0,10.0,12.0,16.0
Capital_Gain,32561.0,1077.648844,7385.292085,0.0,0.0,0.0,0.0,99999.0
Capital_Loss,32561.0,87.303830,402.960219,0.0,0.0,0.0,0.0,4356.0
Hours_per_week,32561.0,40.437456,12.347429,1.0,40.0,40.0,45.0,99.0
Target,32561.0,0.240810,0.427581,0.0,0.0,0.0,0.0,1.0


In [26]:
data_train.dtypes

Age                int64
Workclass         object
fnlwgt             int64
Education         object
Education_Num      int64
Martial_Status    object
Occupation        object
Relationship      object
Race              object
Sex               object
Capital_Gain       int64
Capital_Loss       int64
Hours_per_week     int64
Country           object
Target             int64
dtype: object

As we see, in the test data, age is treated as type **object**. We need to fix this.

In [246]:
data_test['Age'] = data_test['Age'].astype(int)

Also we'll cast all **float** features to **int** type to keep types consistent between our train and test data.

In [247]:
data_test['fnlwgt'] = data_test['fnlwgt'].astype(int)
data_test['Education_Num'] = data_test['Education_Num'].astype(int)
data_test['Capital_Gain'] = data_test['Capital_Gain'].astype(int)
data_test['Capital_Loss'] = data_test['Capital_Loss'].astype(int)
data_test['Hours_per_week'] = data_test['Hours_per_week'].astype(int)

#### Fill in missing data for continuous features with their median values, for categorical features with their mode.

In [251]:
# choose categorical and continuous features from data

categorical_columns = [c for c in data_train.columns 
                       if data_train[c].dtype.name == 'object']
numerical_columns = [c for c in data_train.columns 
                     if data_train[c].dtype.name != 'object']

print('categorical_columns:', categorical_columns)
print('numerical_columns:', numerical_columns)

categorical_columns: ['Workclass', 'Education', 'Martial_Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country']
numerical_columns: ['Age', 'fnlwgt', 'Education_Num', 'Capital_Gain', 'Capital_Loss', 'Hours_per_week', 'Target']


In [252]:
# fill missing data

for c in categorical_columns:
    data_train[c].fillna(data_train[c].mode(), inplace=True)
    data_test[c].fillna(data_train[c].mode(), inplace=True)
    
for c in numerical_columns:
    data_train[c].fillna(data_train[c].median(), inplace=True)
    data_test[c].fillna(data_train[c].median(), inplace=True)

We'll dummy code some categorical features: **Workclass**, **Education**, **Martial_Status**, **Occupation**, **Relationship**, **Race**, **Sex**, **Country**. It can be done via pandas method **get_dummies**

In [253]:
data_train = pd.concat([
    pd.get_dummies(data_train[categorical_columns]),
    data_train[numerical_columns]], axis=1)

data_test = pd.concat([
    pd.get_dummies(data_test[categorical_columns]),
    data_test[numerical_columns]], axis=1)

In [254]:
set(data_train.columns) - set(data_test.columns)

{'Country_ Holand-Netherlands'}

In [255]:
data_train.shape, data_test.shape

((32561, 106), (16281, 105))

#### There is no Holland in the test data. Create new zero-valued feature.

In [256]:
data_test['Country_ Holand-Netherlands'] = 0

In [257]:
set(data_train.columns) - set(data_test.columns)

set()

In [37]:
data_train.head(2)

,Workclass_ Federal-gov,Workclass_ Local-gov,Workclass_ Never-worked,Workclass_ Private,Workclass_ Self-emp-inc,Workclass_ Self-emp-not-inc,Workclass_ State-gov,Workclass_ Without-pay,Education_ 10th,Education_ 11th,...,Country_ United-States,Country_ Vietnam,Country_ Yugoslavia,Age,fnlwgt,Education_Num,Capital_Gain,Capital_Loss,Hours_per_week,Target
0,0,0,0,0,0,0,1,0,0,0,...,1,0,0,39,77516,13,2174,0,40,0
1,0,0,0,0,0,1,0,0,0,0,...,1,0,0,50,83311,13,0,0,13,0


In [38]:
data_test.head(2)

,Workclass_ Federal-gov,Workclass_ Local-gov,Workclass_ Never-worked,Workclass_ Private,Workclass_ Self-emp-inc,Workclass_ Self-emp-not-inc,Workclass_ State-gov,Workclass_ Without-pay,Education_ 10th,Education_ 11th,...,Country_ Vietnam,Country_ Yugoslavia,Age,fnlwgt,Education_Num,Capital_Gain,Capital_Loss,Hours_per_week,Target,Country_ Holand-Netherlands
1,0,0,0,1,0,0,0,0,0,1,...,0,0,25,226802,7,0,0,40,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,38,89814,9,0,0,50,0,0


In [258]:
X_train = data_train.drop(['Target'], axis=1)
y_train = data_train['Target']

X_test = data_test.drop(['Target'], axis=1)
y_test = data_test['Target']

### 3.1 Decision tree without parameter tuning

Train a decision tree **(DecisionTreeClassifier)** with a maximum depth of 3, and evaluate the accuracy metric on the test data. Use parameter **random_state = 17** for results reproducibility.

In [263]:
# you code here
tree = DecisionTreeClassifier(max_depth=3, random_state=17)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf='deprecated', min_samples_split=2,
            min_weight_fraction_leaf='deprecated', presort=False,
            random_state=17, splitter='best')

Make a prediction with the trained model on the test data.

In [264]:
# you code here
tree_predictions = tree.predict(X_test)

In [265]:
# you code here
accuracy_score(y_test, tree_predictions)

0.7904305632332166

6\. What is the test set accuracy of a decision tree with maximum tree depth of 3 and **random_state = 17**?

### 3.2 Decision tree with parameter tuning

Train a decision tree **(DecisionTreeClassifier, random_state = 17).** Find the optimal maximum depth using 5-fold cross-validation **(GridSearchCV)**.

In [266]:
tree_params = {'max_depth': range(2,11)}

locally_best_tree = GridSearchCV(DecisionTreeClassifier(random_state=17), param_grid=tree_params, cv=5)

# you code here                     
locally_best_tree.fit(X_train, y_train)  

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf='deprecated', min_samples_split=2,
            min_weight_fraction_leaf='deprecated', presort=False,
            random_state=17, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': range(2, 11)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [267]:
print("Best params:", locally_best_tree.best_params_)
print("Best cross validaton score", locally_best_tree.best_score_)

Best params: {'max_depth': 9}
Best cross validaton score 0.8561469242345137


Train a decision tree with maximum depth of 9 (it is the best **max_depth** in my case), and compute the test set accuracy. Use parameter **random_state = 17** for reproducibility.

In [269]:
tuned_tree = DecisionTreeClassifier(max_depth=9, random_state=17)
tuned_tree.fit(X_train, y_train)
tuned_tree_predictions = tuned_tree.predict(X_test) # you code here 
accuracy_score(y_test, tuned_tree_predictions) # you code here 

0.7879737116884712

7\. What is the test set accuracy of a decision tree with maximum depth of 9 and **random_state = 17**?

### 3.3 (Optional) Random forest without parameter tuning

Let's take a sneak peek of upcoming lectures and try to use a random forest for our task. For now, you can imagine a random forest as a bunch of decision trees, trained on slightly different subsets of the training data.

Train a random forest **(RandomForestClassifier)**. Set the number of trees to 100 and use **random_state = 17**.

In [46]:
# rf = # you code here 
# rf.fit # you code here 

Make predictions for the test data.

In [47]:
# forest_predictions = rf.predict # you code here 

In [48]:
# accuracy_score # you code here 

### 3.4 (Optional) Random forest with parameter tuning

Train a random forest **(RandomForestClassifier)**. Tune the maximum depth and maximum number of features for each tree using **GridSearchCV**. 

In [49]:
# forest_params = {'max_depth': range(10, 21),
#                 'max_features': range(5, 105, 20)}

# locally_best_forest = GridSearchCV # you code here 

# locally_best_forest.fit # you code here 

In [50]:
# print("Best params:", locally_best_forest.best_params_)
# print("Best cross validaton score", locally_best_forest.best_score_)

Make predictions for the test data.

In [51]:
# tuned_forest_predictions = locally_best_forest.predict # you code here 
# accuracy_score # you code here 